 # HackerEarth Water Day Hackathon

Submission By - Sparsh Saxena

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 11.3 MB/s eta 0:00:00


In [ ]:
# Importing the libraries for use

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error, r2_score
from warnings import filterwarnings
filterwarnings('ignore')
import optuna

In [ ]:
# Connection with the google drive for dataset files

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Importing the train and test datasets into the notebook

train_df = pd.read_csv('/content/gdrive/My Drive/train.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/test.csv')
submission_ids = test_df['Timestamp']

In [ ]:
train_df.head(5)

,Timestamp,Residents,Apartment_Type,Temperature,Humidity,Water_Price,Period_Consumption_Index,Income_Level,Guests,Amenities,Appliance_Usage,Water_Consumption
0,01/01/2002 00,1,Studio,15.31,46.61,1.06,0.97,Low,0,Swimming Pool,0.0,64.85
1,01/01/2002 08,4,NaN,21.01,66.11,2.98,0.91,Upper Middle,1,Swimming Pool,1.0,192.50
2,01/01/2002 16,2,Cottage,12.86,60.86,1.44,1.43,Middle,0,NaN,1.0,116.62
3,02/01/2002 00,2,1BHK,20.16,50.58,1.48,0.91,Middle,-1,Garden,0.0,76.96
4,02/01/2002 08,2,Cottage,16.23,52.25,1.14,1.11,Middle,0,Fountain,0.0,104.70


In [ ]:
test_df.head(5)

,Timestamp,Residents,Apartment_Type,Temperature,Humidity,Water_Price,Period_Consumption_Index,Income_Level,Guests,Amenities,Appliance_Usage
0,11/10/2014 16,5,Bungalow,11.89,57.88,2.77,1.48,Upper Middle,1,Jacuzzi,0.0
1,12/10/2014 00,4,Bungalow,29.22,61.41,2.85,1.30,Upper Middle,0,Garden,NaN
2,12/10/2014 08,3,Cottage,10.27,64.9,1.66,0.97,Middle,0,NaN,0.0
3,12/10/2014 16,3,1BHK,27.03,52.67,1.48,1.31,Low,0,NaN,0.0
4,13/10/2014 00,2,Cottage,12.32,55.23,1.19,1.30,Middle,0,Swimming Pool,0.0


In [ ]:
col_names = train_df.columns
col_names

Index(['Timestamp', 'Residents', 'Apartment_Type', 'Temperature', 'Humidity',
       'Water_Price', 'Period_Consumption_Index', 'Income_Level', 'Guests',
       'Amenities', 'Appliance_Usage', 'Water_Consumption'],
      dtype='object')

In [ ]:
# Convert 'timestamp' column to datetime objects if it's not already
train_df['Timestamp'] = pd.to_datetime(train_df['Timestamp'],format='mixed',dayfirst=True)
test_df['Timestamp'] = pd.to_datetime(test_df['Timestamp'],format='mixed',dayfirst=True)

# Extract year and month
train_df['year'] = train_df['Timestamp'].dt.year
train_df['month'] = train_df['Timestamp'].dt.month
train_df['hour'] = train_df['Timestamp'].dt.hour

test_df['year'] = test_df['Timestamp'].dt.year
test_df['month'] = test_df['Timestamp'].dt.month
test_df['hour'] = test_df['Timestamp'].dt.hour

In [ ]:
train_df.head(5)

,Timestamp,Residents,Apartment_Type,Temperature,Humidity,Water_Price,Period_Consumption_Index,Income_Level,Guests,Amenities,Appliance_Usage,Water_Consumption,year,month,hour
0,2002-01-01 00:00:00,1,Studio,15.31,46.61,1.06,0.97,Low,0,Swimming Pool,0.0,64.85,2002,1,0
1,2002-01-01 08:00:00,4,NaN,21.01,66.11,2.98,0.91,Upper Middle,1,Swimming Pool,1.0,192.50,2002,1,8
2,2002-01-01 16:00:00,2,Cottage,12.86,60.86,1.44,1.43,Middle,0,NaN,1.0,116.62,2002,1,16
3,2002-01-02 00:00:00,2,1BHK,20.16,50.58,1.48,0.91,Middle,-1,Garden,0.0,76.96,2002,1,0
4,2002-01-02 08:00:00,2,Cottage,16.23,52.25,1.14,1.11,Middle,0,Fountain,0.0,104.70,2002,1,8


In [ ]:
test_df.head(5)

,Timestamp,Residents,Apartment_Type,Temperature,Humidity,Water_Price,Period_Consumption_Index,Income_Level,Guests,Amenities,Appliance_Usage,year,month,hour
0,2014-10-11 16:00:00,5,Bungalow,11.89,57.88,2.77,1.48,Upper Middle,1,Jacuzzi,0.0,2014,10,16
1,2014-10-12 00:00:00,4,Bungalow,29.22,61.41,2.85,1.30,Upper Middle,0,Garden,NaN,2014,10,0
2,2014-10-12 08:00:00,3,Cottage,10.27,64.9,1.66,0.97,Middle,0,NaN,0.0,2014,10,8
3,2014-10-12 16:00:00,3,1BHK,27.03,52.67,1.48,1.31,Low,0,NaN,0.0,2014,10,16
4,2014-10-13 00:00:00,2,Cottage,12.32,55.23,1.19,1.30,Middle,0,Swimming Pool,0.0,2014,10,0


In [ ]:
# Checking for null values in train.csv

train_df.isnull().sum()

,0
Timestamp,0
Residents,0
Apartment_Type,426
Temperature,441
Humidity,0
Water_Price,0
Period_Consumption_Index,0
Income_Level,426
Guests,0
Amenities,5997


In [ ]:
# Checking for null values in test.csv

test_df.isnull().sum()

,0
Timestamp,0
Residents,0
Apartment_Type,166
Temperature,150
Humidity,0
Water_Price,0
Period_Consumption_Index,0
Income_Level,165
Guests,0
Amenities,2513


In [ ]:
# Fill missing values in train_df
train_df.fillna({
    'Temperature': train_df['Temperature'].mean(),  # Direct mean calculation
    'Appliance_Usage': train_df['Appliance_Usage'].mean(),
    'Apartment_Type': 'non',
    'Income_Level': 'non',
    'Amenities': 'non'
}, inplace=True)

# Fill missing values in test_df using train_df statistics
test_df.fillna({
    'Temperature': train_df['Temperature'].mean(),
    'Appliance_Usage': train_df['Appliance_Usage'].mean(),  # Fixed column name
    'Apartment_Type': 'non',
    'Income_Level': 'non',
    'Amenities': 'non'
}, inplace=True)

In [ ]:
# Checking out the types of columns

print(train_df.dtypes)

Timestamp                   datetime64[ns]
Residents                            int64
Apartment_Type                      object
Temperature                        float64
Humidity                            object
Water_Price                        float64
Period_Consumption_Index           float64
Income_Level                        object
Guests                               int64
Amenities                           object
Appliance_Usage                    float64
Water_Consumption                  float64
year                                 int32
month                                int32
hour                                 int32
dtype: object


In [ ]:
# List of numerical columns to convert to float64
numerical_cols = ['Residents', 'Guests', 'year', 'month', 'hour']

# List of object columns to convert to float64
float_conversion_cols = ['Temperature', 'Humidity', 'Appliance_Usage']

# Convert numerical columns to float64
train_df[numerical_cols] = train_df[numerical_cols].astype('float64')
test_df[numerical_cols] = test_df[numerical_cols].astype('float64')

# Convert 'Humidity' to numeric, replacing invalid values with NaN
train_df['Humidity'] = pd.to_numeric(train_df['Humidity'], errors='coerce')
test_df['Humidity'] = pd.to_numeric(test_df['Humidity'], errors='coerce')

# Fill NaN values in 'Humidity' with column mean
humidity_mean = train_df['Humidity'].mean()
train_df['Humidity'].fillna(humidity_mean, inplace=True)
test_df['Humidity'].fillna(humidity_mean, inplace=True)

# Convert other object columns to float64 safely
for col in ['Temperature', 'Appliance_Usage']:
    train_df[col] = pd.to_numeric(train_df[col], errors='coerce')
    test_df[col] = pd.to_numeric(test_df[col], errors='coerce')

# Fill NaN values in numeric columns with their respective means
for col in float_conversion_cols:
    train_df[col].fillna(train_df[col].mean(), inplace=True)
    test_df[col].fillna(train_df[col].mean(), inplace=True)  # Using train mean for consistency

# Fill categorical missing values
train_df.fillna({'Apartment_Type': 'non', 'Income_Level': 'non', 'Amenities': 'non'}, inplace=True)
test_df.fillna({'Apartment_Type': 'non', 'Income_Level': 'non', 'Amenities': 'non'}, inplace=True)

# Convert Apartment_Type to object
train_df['Apartment_Type'] = train_df['Apartment_Type'].astype('object')
test_df['Apartment_Type'] = test_df['Apartment_Type'].astype('object')

# Display updated types
print("Train Data Types for columns : \n")
print(train_df.dtypes)

print()

print("Test Data Types for columns : \n")
print(test_df.dtypes)

Train Data Types for columns : 

Timestamp                   datetime64[ns]
Residents                          float64
Apartment_Type                      object
Temperature                        float64
Humidity                           float64
Water_Price                        float64
Period_Consumption_Index           float64
Income_Level                        object
Guests                             float64
Amenities                           object
Appliance_Usage                    float64
Water_Consumption                  float64
year                               float64
month                              float64
hour                               float64
dtype: object

Test Data Types for columns : 

Timestamp                   datetime64[ns]
Residents                          float64
Apartment_Type                      object
Temperature                        float64
Humidity                           float64
Water_Price                        float64
Period_Consumptio

In [ ]:
# Identifying categorical columns
categorical_cols = train_df.select_dtypes(include=['object']).columns

# Applying LabelEncoder to each categorical column
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])  # Transforming the column in train_df
    test_df[col] = le.fit_transform(test_df[col])  # Transforming the column in test_df
    label_encoders[col] = le  # Storing encoder for future inverse transform if needed

In [ ]:
# Converting the int64 columns to float64

int64_cols = train_df.select_dtypes(include=['int64']).columns
for col in int64_cols:
    train_df[col] = train_df[col].astype('float64')
    test_df[col] = test_df[col].astype('float64')

In [ ]:
# Dropping the Timestamp column as it is irrelevant here

train_df.drop('Timestamp', axis=1, inplace=True)
test_df.drop('Timestamp', axis=1, inplace=True)

In [ ]:
# Splitting the dataset into feature matrix and the target column

target = 'Water_Consumption'

X = train_df.drop(target, axis=1)
y = train_df[target]

In [ ]:
# Train Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Scaling the feature matrix using scalers to use them appropriately with tree and non-tree regression models

min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

X_train_mm = min_max_scaler.fit_transform(X_train)
X_train_ss = standard_scaler.fit_transform(X_train)

X_test_mm = min_max_scaler.fit_transform(X_test)
X_test_ss = standard_scaler.fit_transform(X_test)

In [ ]:
# Model Training using various regressor models and their evaluation

analysis_data = []

m1 = KNeighborsRegressor()
m2 = RandomForestRegressor()
m3 = XGBRegressor()
m4 = LinearRegression()
m5 = SVR()

models = {
    'KNN Regressor' : m1,
    'Random Forest Regressor' : m2,
    'XG Boost Regressor' : m3,
    'Linear Regression' : m4,
    'Support Vector Regressor': m5
}

for model_name in models:
  model = models[model_name]

  if model_name not in ('Random Forest Regressor', 'XG Boost Regressor'):
    model.fit(X_train_ss,y_train)
    pred = model.predict(X_test_ss)
  else:
    model.fit(X_train_mm,y_train)
    pred = model.predict(X_test_mm)

  score = max(0,100-np.sqrt(mean_squared_error(y_test,pred)))

  analysis_data.append([model_name, score])

In [ ]:
# Evaluation Results for various regression methods

display(pd.DataFrame(list(sorted(analysis_data,key = lambda x: x[1], reverse=True)),columns=["Model","Score"]))

,Model,Score
0,Random Forest Regressor,69.763665
1,XG Boost Regressor,62.440231
2,KNN Regressor,51.651752
3,Support Vector Regressor,45.675090
4,Linear Regression,38.549817


In [ ]:
# Using Bayesian Optimization to find the best hyperparams


# Split train data further into internal train and validation sets
X_train_opt, X_val, y_train_opt, y_val = train_test_split(X_train_mm, y_train, test_size=0.2, random_state=42)

def objective(trial):
    # Define a thorough search space
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, step=50)
    max_depth = trial.suggest_int('max_depth', 5, 100, step=5)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    criterion = trial.suggest_categorical('criterion', ['squared_error', 'absolute_error'])

    # Train the Random Forest model
    rf = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        bootstrap=bootstrap,
        criterion=criterion,
        random_state=42,
        n_jobs=-1
    )

    rf.fit(X_train_opt, y_train_opt)  # Train on smaller training set
    pred = rf.predict(X_val)  # Validate on internal validation set

    # Compute the custom metric
    mse = mean_squared_error(y_val, pred)
    score = max(0, 100 - np.sqrt(mse))

    return score  # Optuna will try to maximize this

In [ ]:
# Carrying out the study using optimization process for best hyperparams

study = optuna.create_study(direction='maximize')  # Maximize the custom score
study.optimize(objective, n_trials=50)  # Run 50 trials for thorough tuning

# Print best hyperparameters
print("Best Hyperparameters:", study.best_params)

[I 2025-03-24 09:22:42,489] A new study created in memory with name: no-name-24086e69-4087-4eb3-81b3-79a7a65ef381
[I 2025-03-24 09:23:43,448] Trial 0 finished with value: 75.65296615599902 and parameters: {'n_estimators': 200, 'max_depth': 50, 'min_samples_split': 19, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'absolute_error'}. Best is trial 0 with value: 75.65296615599902.
[I 2025-03-24 09:31:20,412] Trial 1 finished with value: 79.98640114709261 and parameters: {'n_estimators': 650, 'max_depth': 75, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False, 'criterion': 'absolute_error'}. Best is trial 1 with value: 79.98640114709261.
[I 2025-03-24 09:44:07,509] Trial 2 finished with value: 77.60956189127407 and parameters: {'n_estimators': 750, 'max_depth': 50, 'min_samples_split': 10, 'min_samples_leaf': 7, 'max_features': None, 'bootstrap': True, 'criterion': 'absolute_error'}. Best is trial 1 with value: 79.986

Best Hyperparameters: {'n_estimators': 550, 'max_depth': 45, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False, 'criterion': 'absolute_error'}


In [ ]:
# Testing model after hyperparam tuning

params = study.best_params
model_test = RandomForestRegressor(
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        min_samples_split=params['min_samples_split'],
        min_samples_leaf=params['min_samples_leaf'],
        max_features=params['max_features'],
        bootstrap=params['bootstrap'],
        criterion=params['criterion'],
        random_state=42,
        n_jobs=-1
)

model_test.fit(X_train_mm, y_train)

RandomForestRegressor(bootstrap=False, criterion='absolute_error', max_depth=45,
                      max_features='sqrt', n_estimators=550, n_jobs=-1,
                      random_state=42)

In [ ]:
# Testing model after hyperparam tuning

predictions = model_test.predict(X_test_mm)
mse = mean_squared_error(y_test, pred)
score = max(0, 100 - np.sqrt(mse))
print("Training Score after tuning : ", score)

Training Score after tuning :  45.675090208743185


In [ ]:
# Training on full train data

X_train = train_df.drop(target, axis=1)
y_train = train_df[target]

X_train_mm = min_max_scaler.fit_transform(X_train)

model_test.fit(X_train_mm, y_train)

RandomForestRegressor(bootstrap=False, criterion='absolute_error', max_depth=45,
                      max_features='sqrt', n_estimators=550, n_jobs=-1,
                      random_state=42)

In [ ]:
X_test = min_max_scaler.fit_transform(test_df)
predicted = pd.DataFrame(model_test.predict(X_test),columns=['Water_Consumption'])
answer = pd.concat([submission_ids,predicted],axis=1)
answer.to_csv('submission.csv',index=False)